In [2]:
import pandas as pd

In [9]:
df = pd.read_csv('./data/journey_plan_2024-08-13_12-51.csv')

# transform from timesterings to datetime
df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
df['end_time'] = pd.to_datetime(df['end_time'], unit='ms')

# # group by id and retunr the total time from the first row start_time to the last row end_time
df['total_time'] = df['end_time'] - df['start_time']
df['total_time'] = df['total_time'].dt.total_seconds()


# add the sum of the total time for each id on each row
df['total_time_sum'] = df.groupby('id')['total_time'].transform('sum')
df

,mode,start_time,end_time,from,to,route,distance,agency,id,duration,transfers,modes,total_time,total_time_sum
0,WALK,2024-08-13 10:51:00,2024-08-13 11:24:53,Origin,Destination,NaN,2437.03,NaN,36f3fd44-733a-4638-a4be-a8ad1612408d,2033,0,['WALK'],2033.0,2033.0
1,WALK,2024-08-13 10:51:59,2024-08-13 10:58:41,Origin,La Sagrera,NaN,552.96,NaN,548b335d-953a-4721-bc4e-1d7bd051e200,1573,0,"['WALK', 'SUBWAY']",402.0,1543.0
2,SUBWAY,2024-08-13 10:58:56,2024-08-13 11:01:34,La Sagrera,Clot,Hospital de Bellvitge - Fondo,1357.41,TMB,548b335d-953a-4721-bc4e-1d7bd051e200,1573,0,"['WALK', 'SUBWAY']",158.0,1543.0
3,WALK,2024-08-13 11:01:49,2024-08-13 11:18:12,Clot,Destination,NaN,1008.75,NaN,548b335d-953a-4721-bc4e-1d7bd051e200,1573,0,"['WALK', 'SUBWAY']",983.0,1543.0
4,WALK,2024-08-13 10:57:35,2024-08-13 11:00:45,Origin,Pl dels Indians,NaN,140.02,NaN,32a78b30-54c9-4272-ab3b-e2b2c7a4fe8a,1542,0,"['WALK', 'BUS']",190.0,1512.0
5,BUS,2024-08-13 11:01:00,2024-08-13 11:10:00,Pl dels Indians,Espronceda - Gran Via,Pg. Marítim / Canyelles,2098.52,TMB,32a78b30-54c9-4272-ab3b-e2b2c7a4fe8a,1542,0,"['WALK', 'BUS']",540.0,1512.0
6,WALK,2024-08-13 11:10:15,2024-08-13 11:23:17,Espronceda - Gran Via,Destination,NaN,848.61,NaN,32a78b30-54c9-4272-ab3b-e2b2c7a4fe8a,1542,0,"['WALK', 'BUS']",782.0,1512.0
7,WALK,2024-08-13 11:07:55,2024-08-13 11:13:21,Origin,Meridiana - La Sagrera,NaN,347.73,NaN,c34f8ce0-3579-45b9-936e-7c151d994ccf,1467,0,"['WALK', 'BUS']",326.0,1437.0
8,BUS,2024-08-13 11:13:36,2024-08-13 11:21:04,Meridiana - La Sagrera,Meridiana - Aragó,<M> Clot / Ciutat Meridiana,1551.03,TMB,c34f8ce0-3579-45b9-936e-7c151d994ccf,1467,0,"['WALK', 'BUS']",448.0,1437.0
9,WALK,2024-08-13 11:21:19,2024-08-13 11:32:22,Meridiana - Aragó,Destination,NaN,694.73,NaN,c34f8ce0-3579-45b9-936e-7c151d994ccf,1467,0,"['WALK', 'BUS']",663.0,1437.0


In [4]:
# get the minimum start_time for each id and the maximum end_time for each id
df_min_max_time = df.groupby('id').agg({'start_time': 'min', 'end_time': 'max', 'total_time': 'sum'}).reset_index()

# calculate teh diff in time between the min and max time
df_min_max_time['diff_time'] = df_min_max_time['end_time'] - df_min_max_time['start_time']

,id,start_time,end_time,total_time,diff_time
0,16f5988a-50b7-4be3-992b-1037c0df0579,2024-08-12 09:00:00,2024-08-12 09:33:53,2033.0,0 days 00:33:53
1,2229a403-822d-43ef-89ad-65d0d14bfd56,2024-08-12 09:00:47,2024-08-12 09:27:00,1543.0,0 days 00:26:13
2,745474ee-125a-4e6a-b7bd-fb5c91676a83,2024-08-12 09:07:05,2024-08-12 09:32:17,1482.0,0 days 00:25:12
3,cba8e6b5-e7e7-4c94-b754-0cd592b7bf8c,2024-08-12 09:07:55,2024-08-12 09:32:22,1437.0,0 days 00:24:27


In [7]:

# return the id with less diff_time
id_less_diff_time = df_min_max_time.loc[df_min_max_time['diff_time'].idxmin(), 'id']

# filter the rows with the id with less diff_time
df_id_less_diff_time = df[df['id'] == id_less_diff_time]

# get the uniwue modes from df
unique_modes = df_id_less_diff_time['mode'].unique()
way_to_go = ' & '.join(unique_modes.tolist())

In [8]:
way_to_go

'WALK & BUS'